In [ ]:
from sentence_transformers.models import LSTM, Dense
from keras import Sequential
import sqlite3
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import random
from collections import Counter
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima.model import ARIMA


# Function to get database connection
def get_db_connection():
    project_root = os.getcwd()
    db_path = os.path.join(project_root, 'disaster_alerts.db')
    conn = sqlite3.connect(db_path)
    conn.row_factory = sqlite3.Row  # Allows accessing columns by name
    return conn

# Fetch data from the database (for demonstration, future predictions are based on historical data)
def fetch_data_from_db():
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT ID, Title, Country, Summary, Keywords, Tags, Date, Priority FROM alerts")
    data = cursor.fetchall()  # Fetch all rows
    conn.close()
    return data

# Convert fetched data into a DataFrame
def prepare_data_for_prediction(data):
    # Extracting 'Date' and 'Priority' to form a time series
    dates = []
    priorities = []
    for row in data:
        try:
            date_value = datetime.strptime(row['Date'], '%Y-%m-%d %H:%M:%S')
            priority_value = int(row['Priority'])
            dates.append(date_value)
            priorities.append(priority_value)
        except (ValueError, KeyError):
            # Skip any rows where 'Date' or 'Priority' are invalid or missing
            continue

    # Check if lists are of the same length
    if len(dates) != len(priorities):
        raise ValueError(f"Date and Priority arrays have different lengths: {len(dates)} != {len(priorities)}")

    # Convert to DataFrame
    df = pd.DataFrame({'Date': dates, 'Priority': priorities})
    df.set_index('Date', inplace=True)

    # Ensure the DataFrame is not empty
    if df.empty:
        raise ValueError("DataFrame is empty after filtering invalid data.")

    return df

# ARIMA prediction model
def arima_model(df, steps=7):
    model = ARIMA(df['Priority'], order=(5, 1, 0))  # ARIMA(p,d,q) model
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=steps)
    return forecast

# LSTM prediction model
def lstm_model(df, steps=7):
    # Preprocessing data for LSTM
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(df['Priority'].values.reshape(-1, 1))

    # Prepare the data for LSTM (using the last 60 data points to predict the next)
    x_data, y_data = [], []
    for i in range(60, len(data_scaled)):
        x_data.append(data_scaled[i-60:i, 0])
        y_data.append(data_scaled[i, 0])

    x_data, y_data = np.array(x_data), np.array(y_data)
    x_data = np.reshape(x_data, (x_data.shape[0], x_data.shape[1], 1))

    # Define the LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(x_data.shape[1], 1)))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(x_data, y_data, epochs=5, batch_size=32)

    # Make predictions
    predicted = []
    last_60_days = data_scaled[-60:].reshape(1, -1)
    last_60_days = np.reshape(last_60_days, (last_60_days.shape[0], last_60_days.shape[1], 1))

    for _ in range(steps):
        predicted_value = model.predict(last_60_days)
        predicted.append(predicted_value[0][0])
        last_60_days = np.append(last_60_days[:, 1:, :], predicted_value.reshape(1, 1, 1), axis=1)

    # Inverse scaling to get actual values
    predicted = scaler.inverse_transform(np.array(predicted).reshape(-1, 1))
    return predicted.flatten()

# Generate future predictions (simulated here)
def generate_future_predictions(duration=7):
    future_dates = [datetime.now() + timedelta(days=i) for i in range(1, duration + 1)]  # Next 7 days for short-term
    future_predictions = [random.randint(50, 100) for _ in range(duration)]  # Simulated prediction data
    return future_dates, future_predictions

# Generate long-term predictions for the next 6 months
def generate_long_term_predictions(months=6):
    future_months = [datetime.now() + timedelta(weeks=4 * i) for i in range(1, months + 1)]  # Next 6 months
    future_month_predictions = [random.randint(60, 100) for _ in range(months)]  # Simulated predictions
    return future_months, future_month_predictions

# Plot charts
def plot_charts(future_dates, future_predictions, future_months, future_month_predictions, data):
    # 1. Prediction Timeline (Line Chart)
    plt.figure(figsize=(10, 6))
    plt.plot(future_dates, future_predictions, label="Prediction for Next 7 Days", color='teal', marker='o')
    plt.title('Prediction Timeline (Next 7 Days)')
    plt.xlabel('Date')
    plt.ylabel('Prediction (%)')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # 2. Bar Chart: Short-Term Predictions (Today, Tomorrow, Day after)
    bar_dates = [future_dates[i].strftime('%Y-%m-%d') for i in range(len(future_dates))]

    plt.figure(figsize=(10, 6))
    plt.bar(bar_dates, future_predictions, color='purple', alpha=0.6)
    plt.title('Short-Term Predictions (Next 7 Days)')
    plt.xlabel('Date')
    plt.ylabel('Prediction (%)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # 3. Long-Term Predictions (6 months prediction)
    long_term_dates = [future_months[i].strftime('%Y-%m') for i in range(len(future_months))]

    plt.figure(figsize=(10, 6))
    plt.plot(long_term_dates, future_month_predictions, label="Prediction for Next 6 Months", color='blue', marker='x')
    plt.title('Long-Term Prediction (Next 6 Months)')
    plt.xlabel('Month')
    plt.ylabel('Prediction Accuracy (%)')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # 4. Pie Chart: Keyword Distribution (Predicted)
    keywords = []
    for row in data:
        if row['Keywords']:
            keywords.extend(row['Keywords'].split(','))

    keyword_count = Counter(keywords)
    top_keywords = dict(keyword_count.most_common(5))  # Top 5 most frequent keywords

    plt.figure(figsize=(8, 8))
    plt.pie(top_keywords.values(), labels=top_keywords.keys(), autopct='%1.1f%%', startangle=90,
            colors=['#FF6384', '#36A2EB', '#FFCE56', '#FF5733', '#C70039'])
    plt.title('Top 5 Predicted Keywords Distribution')
    plt.tight_layout()
    plt.show()

# Function to display Tags and Accuracy
def display_tags_and_accuracy(data, future_predictions):
    # 1. Display the Tags for each alert
    print("\nTags associated with the alerts:\n")
    tags = set()
    for row in data:
        if row['Tags']:
            tags.update(row['Tags'].split(','))  # Collect unique tags
    print(f"Unique Tags: {', '.join(tags)}\n")

    # 2. Display a simulated accuracy based on the predictions
    print("\nPrediction Accuracy Display:\n")
    accuracy = random.uniform(85, 95)  # Simulated prediction accuracy (between 85% to 95%)
    print(f"Simulated Prediction Accuracy: {accuracy:.2f}%\n")

# Main execution flow
if __name__ == '__main__':
    # Fetch data from the database (for historical context)
    data = fetch_data_from_db()

    try:
        # Prepare data for prediction
        df = prepare_data_for_prediction(data)

        # Generate predictions using ARIMA
        arima_forecast = arima_model(df, steps=7)
        print("\nARIMA Forecast for Next 7 Days:")
        print(arima_forecast)

        # Generate predictions using LSTM
        lstm_forecast = lstm_model(df, steps=7)
        print("\nLSTM Forecast for Next 7 Days:")
        print(lstm_forecast)

    except ValueError as e:
        print(f"Error: {e}")

    # Generate short-term predictions for the next 7 days (simulated)
    future_dates, future_predictions = generate_future_predictions(duration=7)

    # Generate long-term predictions for the next 6 months (simulated)
    future_months, future_month_predictions = generate_long_term_predictions(months=6)

    # Plot the charts based on future predictions
    plot_charts(future_dates, future_predictions, future_months, future_month_predictions, data)

    # Display Tags and simulated Accuracy
    display_tags_and_accuracy(data, future_predictions)


In [ ]:
import sqlite3
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import random
from collections import Counter
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima.model import ARIMA


# Function to get database connection
def get_db_connection():
    project_root = os.getcwd()
    db_path = os.path.join(project_root, 'disaster_alerts.db')
    conn = sqlite3.connect(db_path)
    conn.row_factory = sqlite3.Row  # Allows accessing columns by name
    return conn

# Fetch data from the database (only for india)
def fetch_data_from_db(country='india'):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT ID, Title, Country, Summary, Keywords, Tags, Date, Priority FROM alerts WHERE Country=?", (country,))
    data = cursor.fetchall()  # Fetch all rows where the country is india
    conn.close()
    return data

# Convert fetched data into a DataFrame
def prepare_data_for_prediction(data):
    # Extracting 'Date' and 'Priority' to form a time series
    dates = []
    priorities = []
    for row in data:
        try:
            date_value = datetime.strptime(row['Date'], '%Y-%m-%d %H:%M:%S')
            priority_value = int(row['Priority'])
            dates.append(date_value)
            priorities.append(priority_value)
        except (ValueError, KeyError):
            # Skip any rows where 'Date' or 'Priority' are invalid or missing
            continue

    # Check if lists are of the same length
    if len(dates) != len(priorities):
        raise ValueError(f"Date and Priority arrays have different lengths: {len(dates)} != {len(priorities)}")

    # Convert to DataFrame
    df = pd.DataFrame({'Date': dates, 'Priority': priorities})
    df.set_index('Date', inplace=True)

    # Ensure the DataFrame is not empty
    if df.empty:
        raise ValueError("DataFrame is empty after filtering invalid data.")

    return df

# ARIMA prediction model
def arima_model(df, steps=7):
    model = ARIMA(df['Priority'], order=(5, 1, 0))  # ARIMA(p,d,q) model
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=steps)
    return forecast

# LSTM prediction model
def lstm_model(df, steps=7):
    # Preprocessing data for LSTM
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(df['Priority'].values.reshape(-1, 1))

    # Prepare the data for LSTM (using the last 60 data points to predict the next)
    x_data, y_data = [], []
    for i in range(60, len(data_scaled)):
        x_data.append(data_scaled[i-60:i, 0])
        y_data.append(data_scaled[i, 0])

    x_data, y_data = np.array(x_data), np.array(y_data)
    x_data = np.reshape(x_data, (x_data.shape[0], x_data.shape[1], 1))

    # Define the LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(x_data.shape[1], 1)))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(x_data, y_data, epochs=5, batch_size=32)

    # Make predictions
    predicted = []
    last_60_days = data_scaled[-60:].reshape(1, -1)
    last_60_days = np.reshape(last_60_days, (last_60_days.shape[0], last_60_days.shape[1], 1))

    for _ in range(steps):
        predicted_value = model.predict(last_60_days)
        predicted.append(predicted_value[0][0])
        last_60_days = np.append(last_60_days[:, 1:, :], predicted_value.reshape(1, 1, 1), axis=1)

    # Inverse scaling to get actual values
    predicted = scaler.inverse_transform(np.array(predicted).reshape(-1, 1))
    return predicted.flatten()

# Generate future predictions (simulated here) for india
def generate_future_predictions(duration=7):
    future_dates = [datetime.now() + timedelta(days=i) for i in range(1, duration + 1)]  # Next 7 days for short-term
    future_predictions = [random.randint(50, 100) for _ in range(duration)]  # Simulated prediction data
    return future_dates, future_predictions

# Generate long-term predictions for the next 6 months (simulated)
def generate_long_term_predictions(months=6):
    future_months = [datetime.now() + timedelta(weeks=4 * i) for i in range(1, months + 1)]  # Next 6 months
    future_month_predictions = [random.randint(60, 100) for _ in range(months)]  # Simulated predictions
    return future_months, future_month_predictions

# Plot charts
def plot_charts(future_dates, future_predictions, future_months, future_month_predictions, data):
    # 1. Prediction Timeline (Line Chart)
    plt.figure(figsize=(10, 6))
    plt.plot(future_dates, future_predictions, label="Prediction for Next 7 Days", color='teal', marker='o')
    plt.title('Prediction Timeline (Next 7 Days)')
    plt.xlabel('Date')
    plt.ylabel('Prediction (%)')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # 2. Bar Chart: Short-Term Predictions (Today, Tomorrow, Day after)
    bar_dates = [future_dates[i].strftime('%Y-%m-%d') for i in range(len(future_dates))]

    plt.figure(figsize=(10, 6))
    plt.bar(bar_dates, future_predictions, color='purple', alpha=0.6)
    plt.title('Short-Term Predictions (Next 7 Days)')
    plt.xlabel('Date')
    plt.ylabel('Prediction (%)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # 3. Long-Term Predictions (6 months prediction)
    long_term_dates = [future_months[i].strftime('%Y-%m') for i in range(len(future_months))]

    plt.figure(figsize=(10, 6))
    plt.plot(long_term_dates, future_month_predictions, label="Prediction for Next 6 Months", color='blue', marker='x')
    plt.title('Long-Term Prediction (Next 6 Months)')
    plt.xlabel('Month')
    plt.ylabel('Prediction Accuracy (%)')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # 4. Pie Chart: Keyword Distribution (Predicted)
    keywords = []
    for row in data:
        if row['Keywords']:
            keywords.extend(row['Keywords'].split(','))

    keyword_count = Counter(keywords)
    top_keywords = dict(keyword_count.most_common(5))  # Top 5 most frequent keywords

    plt.figure(figsize=(8, 8))
    plt.pie(top_keywords.values(), labels=top_keywords.keys(), autopct='%1.1f%%', startangle=90,
            colors=['#FF6384', '#36A2EB', '#FFCE56', '#FF5733', '#C70039'])
    plt.title('Top 5 Predicted Keywords Distribution ')
    plt.tight_layout()
    plt.show()

# Function to display Tags and Accuracy
def display_tags_and_accuracy(data, future_predictions):
    # 1. Display the Tags for each alert
    print("\nTags associated with the alerts:\n")
    tags = set()
    for row in data:
        if row['Tags']:
            tags.update(row['Tags'].split(','))  # Collect unique tags
    print(f"Unique Tags: {', '.join(tags)}\n")

    # 2. Display a simulated accuracy based on the predictions
    print("\nPrediction Accuracy Display:\n")
    accuracy = random.uniform(85, 95)  # Simulated prediction accuracy (between 85% to 95%)
    print(f"Simulated Prediction Accuracy: {accuracy:.2f}%\n")

# Main execution flow
if __name__ == '__main__':
    # Fetch data for india
    data = fetch_data_from_db(country='india')

    try:
        # Prepare data for prediction
        df = prepare_data_for_prediction(data)

        # Generate predictions using ARIMA
        arima_forecast = arima_model(df, steps=7)
        print("\nARIMA Forecast for Next 7 Days:")
        print(arima_forecast)

        # Generate predictions using LSTM
        lstm_forecast = lstm_model(df, steps=7)
        print("\nLSTM Forecast for Next 7 Days:")
        print(lstm_forecast)

    except ValueError as e:
        print(f"Error: {e}")

    # Generate short-term predictions for the next 7 days (simulated)
    future_dates, future_predictions = generate_future_predictions(duration=7)

    # Generate long-term predictions for the next 6 months (simulated)
    future_months, future_month_predictions = generate_long_term_predictions(months=6)

    # Plot the charts based on future predictions
    plot_charts(future_dates, future_predictions, future_months, future_month_predictions, data)

    # Display Tags and simulated Accuracy
    display_tags_and_accuracy(data, future_predictions)
